# **Limpeza de Dados do Data Set Tips**

## Índice:
- [Visão Geral do Projeto](#geral)
- [Carregamento dos Dados](#pt01)
- [Tratamento de Valores Ausentes](#pt02)
- [Detecção e Tratamento de Outliers](#pt03)
- [Transformação e Codificação de Dados](#pt04)
- [Verificação de Dados Transformados](#pt05)
- [Conclusão](#conclusao)


## **Visão Geral**<a id="geral"></a>
Este notebook apresenta a limpeza e pré-processamento de dados do data set `Tips`. O objetivo é preparar os dados para análises subsequentes, garantindo sua qualidade e adequação para modelos de aprendizado de máquina.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

## **1. Carregamento dos Dados**<a id="pt01"></a>
* O conjunto de dados tips_data.csv é carregado usando a biblioteca pandas.
* As primeiras linhas e um resumo das colunas são exibidos para uma visão geral dos dados.
* Um dicionário de dados tips_dictionary.csv fornece informações sobre o tipo e significado de cada variável.

In [2]:
import pandas as pd

# Carregar os dados
df = pd.read_csv('../data/raw/tips_data.csv')
# Exibir as primeiras linhas dos dados
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
dicionario = pd.read_csv('../data/external/tips_dictionary.csv')
dicionario

,variavel,descricao,tipo,subtipo
0,total_bill,valor da conta em dólares,quantitativa,contínua
1,tip,valor da gorgeta em dólares,quantitativa,contínua
2,sex,sexo de quem pagou a conta,qualitativa,nominal
3,smoker,Se havia fumantes na mesa,qualitativa,nominal
4,day,dia da semana,qualitativa,ordinal
5,time,hora do dia,qualitativa,nominal
6,size,quantidade de pessoas na mesa,quantitativa,discreta


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


In [5]:
display(df.describe(include='all'))

,total_bill,tip,sex,smoker,day,time,size
count,244.000000,244.000000,244,244,244,244,244.000000
unique,NaN,NaN,2,2,4,2,NaN
top,NaN,NaN,Male,No,Sat,Dinner,NaN
freq,NaN,NaN,157,151,87,176,NaN
mean,19.785943,2.998279,NaN,NaN,NaN,NaN,2.569672
std,8.902412,1.383638,NaN,NaN,NaN,NaN,0.951100
min,3.070000,1.000000,NaN,NaN,NaN,NaN,1.000000
25%,13.347500,2.000000,NaN,NaN,NaN,NaN,2.000000
50%,17.795000,2.900000,NaN,NaN,NaN,NaN,2.000000
75%,24.127500,3.562500,NaN,NaN,NaN,NaN,3.000000


## **2. Tratamento de Valores Ausentes**<a id="pt02"></a>
* Verifica-se a presença de valores ausentes em cada coluna.
* neste notebook, não há valores ausentes, mas o código demonstra como lidar com eles caso existissem.

In [6]:
valores_faltantes = df.isnull().sum()
print("\nValores Faltantes por Coluna:")
print(valores_faltantes)


Valores Faltantes por Coluna:
total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64


## **3. Detecção e Tratamento de Outliers**<a id="pt03"></a>

* Outliers são identificados nas colunas numéricas usando o método do Intervalo Interquartil (IQR).
* Uma classe customizada Lim_Val_Discrepantes é definida para limitar outliers a percentis específicos, se necessário.

In [7]:
df_numerico = df.select_dtypes(include=[np.number])
Q1 = df_numerico.quantile(0.25)
Q3 = df_numerico.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [8]:
outliers = ((df_numerico < lower_bound) | (df_numerico > upper_bound)).sum()
print("\nOutliers por Coluna:")
print(outliers)


Outliers por Coluna:
total_bill    9
tip           9
size          9
dtype: int64


In [9]:
class Lim_Val_Discrepantes(BaseEstimator, TransformerMixin):
   
    def __init__(self, percentil_inferior=5, percentil_superior=95):

        self.percentil_inferior = percentil_inferior
        self.percentil_superior = percentil_superior

    def fit(self, X, y=None):

        return self

    def transform(self, X):

        X_limitado = X.copy()  
        for coluna in range(X_limitado.shape[1]): 
            limite_inferior = np.percentile(X_limitado[:, coluna], self.percentil_inferior)
            limite_superior = np.percentile(X_limitado[:, coluna], self.percentil_superior)

            X_limitado[:, coluna] = np.clip(X_limitado[:, coluna], limite_inferior, limite_superior)

        return X_limitado

## **4. Transformação e Codificação de Dados**<a id="pt04"></a>

* Variáveis categóricas são convertidas para o tipo category.
* A variável ordinal day é codificada com rótulos numéricos que preservam a ordem.
* Variáveis categóricas nominais são codificadas usando One-Hot Encoding.
* Colunas numéricas são padronizadas (escalonadas) usando StandardScaler.
* Um ColumnTransformer combina todas as transformações em um único pipeline

In [18]:
colunas_nominais = ['sex', 'smoker', 'time'] 
df[colunas_nominais] = df[colunas_nominais].astype('category')

ordem_dias = ['Thur', 'Fri', 'Sat', 'Sun']
df['day'] = pd.Categorical(df['day'], categories=ordem_dias, ordered=True)

preprocesso_ordinal = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=[ordem_dias])) 
])

preprocesso_numerico = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')) 
])

colunas_numericas = ['total_bill', 'tip', 'size'] 

transformador_numerico = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocesso = ColumnTransformer([
    ('ordinal', preprocesso_ordinal, ['day']),
    ('numerical', transformador_numerico, colunas_numericas),
    ('nominal', preprocesso_numerico, colunas_nominais)
])

X_transformado = preprocesso.fit_transform(df) 

## **Verificação dos Dados Transformados**<a id="pt05"></a>

* Verifica-se novamente se ainda existem valores ausentes após o pré-processamento.
* Outliers remanescentes são detectados usando o método Z-score.

In [19]:
# 5. Verifying the Transformed Data

# 5.1. Check for Remaining Missing Values
valores_faltantes = np.isnan(X_transformado).sum()
print("\nValores faltantes após pré-processamento:")
print(valores_faltantes)

def detectar_outliers(data, threshold=3):
    z_scores = np.abs(stats.zscore(data))
    outliers = (z_scores > threshold).sum()
    return outliers

nomes_caracteristicas_numericas = preprocesso.transformers_[1][2]

# Get all transformed column names
nomes_caracteristicas_novos = preprocesso.get_feature_names_out()

# Find the indices of numerical features in the transformed data
numerical_feature_indices = [i for i, name in enumerate(nomes_caracteristicas_novos) if any(num_feature in name for num_feature in nomes_caracteristicas_numericas)]

# Apply outlier detection on the numerical features
outliers_restantes = detectar_outliers(X_transformado[:, numerical_feature_indices])

print("\nOutliers Restantes após Pré-processamento:")
print(outliers_restantes)


Valores faltantes após pré-processamento:
0

Outliers Restantes após Pré-processamento:
11


## **Conclusão**

* Este notebook executa a limpeza e preparação de dados, lida com valores ausentes, outliers e codificação de variáveis categóricas. O uso de pipelines do scikit-learn tornou o processo mais organizado e eficiente.